# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113146e+02     1.482029e+00
 * time: 0.6992490291595459
     1     1.071696e+01     9.491443e-01
 * time: 2.0883591175079346
     2    -1.138461e+01     1.105409e+00
 * time: 2.7834880352020264
     3    -3.402291e+01     7.557699e-01
 * time: 3.7636759281158447
     4    -4.737597e+01     5.889780e-01
 * time: 4.72720193862915
     5    -5.702895e+01     2.395512e-01
 * time: 5.724928140640259
     6    -5.997158e+01     1.423997e-01
 * time: 6.382604122161865
     7    -6.101207e+01     4.509268e-02
 * time: 7.047425031661987
     8    -6.138645e+01     7.900208e-02
 * time: 7.70560097694397
     9    -6.164324e+01     3.691503e-02
 * time: 8.371928930282593
    10    -6.184017e+01     2.181680e-02
 * time: 9.027969121932983
    11    -6.199117e+01     1.860231e-02
 * time: 9.675604104995728
    12    -6.205072e+01     1.413046e-02
 * time: 10.32085394859314
    13    -6.211534e+01     1.254323e-02
 * time: 10.997547149658203
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671053
    AtomicLocal         -18.8557599
    AtomicNonlocal      14.8522599
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485343 
    Xc                  -19.3336809

    total               -62.261666454116
